# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [2]:
?VOCS

Init signature:
VOCS(
    *,
    variables: Dict[str, types.ConstrainedListValue] = None,
    constraints: Dict[str, types.ConstrainedListValue] = None,
    objectives: Dict[str, xopt.vocs.ObjectiveEnum] = None,
    constants: Dict[str, Any] = None,
    linked_variables: Dict[str, str] = None,
) -> None
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/Code/GitHub/Xopt/xopt/vocs.py
Type:           ModelMetaclass
Subclasses:     


In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables=None)

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': None}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs


True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": null}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'definitions': {'ConstraintEnum': {'title': 'ConstraintEnum',
   'description': 'An enumeration.',
   'enum': ['LESS_THAN', 'GREATER_THAN'],
   'type': 'string'},
  'ObjectiveEnum': {'title': 'ObjectiveEnum',
   'description': 'An enumeration.',


# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data['c']  = data['a'] + data['b']
data['d']  = data['a'] - data['b']
data['e']  = data['a']*2 + data['b']*2
data['f']  = data['a']*2 - data['b']*2
data.index = np.arange(len(data)) + 5 # custom index
data


,a,b,g,c,d,e,f
5,248.646296,0.278516,1234,248.924812,248.367781,497.849623,496.735561
6,667.959347,0.262053,1234,668.221401,667.697294,1336.442802,1335.394588
7,436.455646,-0.819176,1234,435.636470,437.274822,871.272941,874.549644
8,78.031374,-0.381809,1234,77.649565,78.413183,155.299130,156.826365
9,291.909342,0.167741,1234,292.077083,291.741601,584.154166,583.483201
10,523.538777,0.342438,1234,523.881214,523.196339,1047.762429,1046.392678
11,973.544650,-0.573842,1234,972.970808,974.118492,1945.941615,1948.236984
12,681.645722,0.454722,1234,682.100444,681.191000,1364.200888,1362.381999
13,745.597933,-0.375671,1234,745.222262,745.973605,1490.444524,1491.947209
14,858.613898,-0.159212,1234,858.454686,858.773110,1716.909371,1717.546219


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-248.924812,248.367781
6,-668.221401,667.697294
7,-435.636470,437.274822
8,-77.649565,78.413183
9,-292.077083,291.741601
10,-523.881214,523.196339
11,-972.970808,974.118492
12,-682.100444,681.191000
13,-745.222262,745.973605
14,-858.454686,858.773110


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-248.924812,248.367781
6,-668.221401,667.697294
7,-435.636470,437.274822
8,-77.649565,78.413183
9,-292.077083,291.741601
10,-523.881214,523.196339
11,-972.970808,974.118492
12,-682.100444,681.191000
13,-745.222262,745.973605
14,-858.454686,858.773110


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-248.92481173,  248.36778073],
       [-668.22140096,  667.697294  ],
       [-435.63647041,  437.27482212],
       [ -77.64956512,   78.41318264],
       [-292.07708298,  291.74160052],
       [-523.88121432,  523.19633884],
       [-972.97080758,  974.11849183],
       [-682.10044415,  681.19099974],
       [-745.2222621 ,  745.97360461],
       [-858.45468555,  858.77310975]])

In [17]:
?form_constraint_data

Signature: form_constraint_data(constraints: Dict, data, prefix='constraint_')
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe)
A constraint is satisfied if the evaluation is < 0.

Returns a dataframe with the constraint data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,495.849623,-496.735561
6,1334.442802,-1335.394588
7,869.272941,-874.549644
8,153.299130,-156.826365
9,582.154166,-583.483201
10,1045.762429,-1046.392678
11,1943.941615,-1948.236984
12,1362.200888,-1362.381999
13,1488.444524,-1491.947209
14,1714.909371,-1717.546219


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(constraints: Dict, data, prefix='feasible_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False
